In [11]:
!pip install deepface

In [12]:
import os
import cv2
from deepface import DeepFace
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from matplotlib import pyplot as plt

In [13]:
#-Airan: Parametrización

SIZE_DEFAULT = (143, 181) # Tamaño por defecto de las imagenes.
SIZE_TRANSFORM = [(143, 181), (71, 90), (35, 45), (17, 22)] #143*181 - 71*90 - 35*45 - 17*22

In [14]:
def process_data(size, path): # Default
    print('INICIANDO PROCESADO. Tamaño de procesamiento:',size)
    #-Airan: Información y muestra 1º elemento
    first = True
    elemento = 'NaN'
    #-Airán: Contador de elementos procesados
    elementos_procesados = 0
    
    sub_paths = ['Train', 'Validation'] #-Airan: Sub_Path

    y = []
    y_pred = []
    
    y_train = []
    y_pred_train = []
    
    y_validation = []
    y_pred_validation = []

    # Carpetas con muestras
    folders = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise' ]
    # Etiquetas del clasificador
    classlabels = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise' ]

    # Cada carpeta un índice de clase numérico
    n_clases = 0
    for fex in folders:

        for sub_path in sub_paths: #-Airan: Obtener todo el dataset: Train, Validation

            folder = os.path.join((path + sub_path), fex)

            for file_name in os.listdir(folder):

                # Asume imágenes en formato png
                if file_name.endswith('.png'):
                    
                    abs_path = os.path.join(folder, file_name) #-Airan: Ruta completa imagen
                    
                    elementos_procesados += 1
                    
                    #-Airán: Información y muestra 1º elemento
                    if first:
                        elemento = abs_path
                        first = False                        
                    
                    #-Airan: Lee la imagen
                    image = cv2.imread(abs_path)
                    
                    #-Airan: Modificación tamaño imagen
                    if size != SIZE_DEFAULT: # Simulación diferente a la default
                        image = cv2.resize(image, size) # Airán

                    # Procesa la imagen que asume hay cara, no fuerza la detección
                    # Añadimos directamente la imagen. DeepFace, admite tanto imagen como path.
                    obj = DeepFace.analyze(image, enforce_detection=False, actions = ['emotion']) #-Airan: Image

                    #--- Conjunto completo.
                    y.append(n_clases) # Anotación, índice de la carpeta procesando
                    y_pred.append(classlabels.index(obj["dominant_emotion"])) # Predicción en formato numérico
                    #--- Conjunto Train.
                    if sub_path == sub_paths[0]:
                        y_train.append(n_clases)
                        y_pred_train.append(classlabels.index(obj["dominant_emotion"]))
                    #--- Conjunto Validation.
                    if sub_path == sub_paths[1]:
                        y_validation.append(n_clases)
                        y_pred_validation.append(classlabels.index(obj["dominant_emotion"]))
                        
                    
        n_clases += 1
        
    print('\nINFORMACIÓN DATASET')
    print('Número de carpetas:', len(folders))
    print('Número de clases:', len(classlabels))
    img = cv2.imread(elemento)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    if size != SIZE_DEFAULT: # Simulación diferente a la default
        img = cv2.resize(img, size)
    plt.imshow(img)
    plt.show()
    print('Tamaño de la imagen:',size)
    print('Elementos procesados:',elementos_procesados)
    
    #--- Información de los resultados obtenidos:

    print("\nDeepFace FER Metrics") # Reconocimiento Facial de Emociones.
    
    #--- Información Conjunto Completo:
    print("\nInformación Conjunto Completo")
    precision_score(y, y_pred, average='weighted')
    recall_score(y, y_pred, average='weighted')
    print(classification_report(y, y_pred, target_names=classlabels))
    print("Confussion matrix:")
    print(confusion_matrix(y, y_pred, labels=range(n_clases)))
    #--- Información Conjunto Train:
    print("\nInformación Conjunto Train")
    precision_score(y_train, y_pred_train, average='weighted')
    recall_score(y_train, y_pred_train, average='weighted')
    print(classification_report(y_train, y_pred_train, target_names=classlabels))
    print("Confussion matrix:")
    print(confusion_matrix(y_train, y_pred_train, labels=range(n_clases)))
    #--- Información Conjunto Validation:
    print("\nInformación Conjunto Validation")
    precision_score(y_validation, y_pred_validation, average='weighted')
    recall_score(y_validation, y_pred_validation, average='weighted')
    print(classification_report(y_validation, y_pred_validation, target_names=classlabels))
    print("Confussion matrix:")
    print(confusion_matrix(y_validation, y_pred_validation, labels=range(n_clases)))
    print('FIN PROCESADO')

In [15]:
#-SIMULATIONS:
print('--- INICIO SIMULACIÓN ---')
count = 1
for size in SIZE_TRANSFORM:
    print('\nSimulación:',count)
    process_data(size, '../input/DataSetsSFEW/SFEW/')
    count += 1
print('--- FIN SIMULACIÓN ---')